## Mnist

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch import nn, optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
train_data = datasets.MNIST(root='/Users/qi/Documents/cood/ML/dataBase/',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

test_data = datasets.MNIST(root='/Users/qi/Documents/cood/ML/dataBase/',
                            train=False,
                            transform=transforms.ToTensor(),
                            download=True)

In [3]:
batch_size = 64

In [5]:
train_loader = DataLoader(dataset=train_data,
                          shuffle=True,
                          batch_size=batch_size)

test_loader = DataLoader(dataset=test_data,
                          shuffle=True,
                          batch_size=batch_size)

### CNN

In [13]:
class cnnNet(nn.Module):
    def __init__(self):
        super(cnnNet, self).__init__()
        # 28 * 28
        self.conv1 = nn.Sequential(nn.Conv2d(1,32,5,1,2), nn.ReLU(), nn.MaxPool2d(2,2))
        # 14 * 14 because pooling
        self.conv2 = nn.Sequential(nn.Conv2d(32,64,5,1,2), nn.ReLU(), nn.MaxPool2d(2,2))
        # 7 * 7 because pooling
        self.fc1 = nn.Sequential(nn.Linear(64*7*7,1000), nn.Dropout(0.5), nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(1000,10), nn.Softmax(dim=1))
        
    def forward(self, x):
        # x -> [64,1,28,28]
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [48]:
LR = 0.01
model = cnnNet()
Loss_fc = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), LR)

In [49]:
def train():
    for i, data in enumerate(train_loader):
        inputs, target = data
        out = model(inputs)
        loss = Loss_fc(out, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [50]:
def test():
    corrent = 0
    for i, data in enumerate(test_loader):
        inputs, target = data
        out = model(inputs)
        _, pred = torch.max(out, 1)
        corrent += (pred == target).sum(0)
    print('Test acc {0}'.format(corrent.item()/len(test_data)))

In [51]:
for epoch in range(3):
    print('epoch ',epoch)
    train()
    test()

epoch  0
Test acc 0.1782
epoch  1
Test acc 0.2507
epoch  2
Test acc 0.7721


### LSTM

In [55]:
class lstmNet(nn.Module):
    def __init__(self):
        super(lstmNet, self).__init__()
        # 28 * 28
        self.lstm = nn.LSTM(
            # 输入特征大小 每行28个值
            input_size=28,
            # LSTM模块的数量
            hidden_size=64,
            # 隐藏层数
            num_layers=1,
            # 默认输入是 (seq_len, batch, feature) -> 序列长度、batch、特征
            # 如果 batch_first=True -> (batch,seq_len, feature)
            batch_first=True)
        self.out = nn.Linear(64,10)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        # x -> [64,1,28,28]
        x = x.view(-1,28,28)
        # output 输出数据 -> (batch, seq_len, hidden_size) 包含每一个序列的输出结果
        # h_n 输出数据 -> (num_layers, batch, hidden_size) 包含最后一个序列的输出结果
        # c_n 输出数据 -> (num_layers, batch, hidden_size) 包含最后一个序列的输出结果
        output, (h_n, c_n) = self.lstm(x)
        output_in_lats_timestep = h_n[-1,:,:]
        x = self.out(output_in_lats_timestep)
        x = self.softmax(x)
        return x

In [56]:
LR = 0.01
model = lstmNet()
Loss_fc = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), LR)

In [57]:
for epoch in range(5):
    print('epoch ',epoch)
    train()
    test()

epoch  0
Test acc 0.8912
epoch  1
Test acc 0.9196
epoch  2
Test acc 0.9206
epoch  3
Test acc 0.9322
epoch  4
Test acc 0.9186


### 模型的保存和载入

保存

In [58]:
torch.save(model.state_dict(), 'save_my_model.pth')

载入，先定义模型，再载入

In [59]:
# 定义模型
model_ = lstmNet()
# 加载参数
model_.load_state_dict(torch.load('save_my_model.pth'))

LR = 0.01
Loss_fc = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), LR)

In [60]:
def test_():
    corrent = 0
    for i, data in enumerate(test_loader):
        inputs, target = data
        out = model_(inputs)
        _, pred = torch.max(out, 1)
        corrent += (pred == target).sum(0)
    print('Test acc {0}'.format(corrent.item()/len(test_data)))

In [61]:
test_()

Test acc 0.9186


可以看到已经载入成功了